# Notebook 2 : Silver 1 - Nettoyage (Spark SQL)

**Durée** : 15 minutes  
**Lakehouse** : Lakehouse_silver  
**Objectif** : Nettoyer les données et identifier les limites de SQL

## Cellule 1 : Chargement depuis Bronze

In [ ]:
%%sql
SELECT * FROM bronze.consumption_raw LIMIT 10

## Cellule 2 : Suppression des doublons

In [ ]:
%%sql
CREATE OR REPLACE TEMP VIEW consumption_dedup AS
SELECT DISTINCT * FROM bronze.consumption_raw

## Cellule 3 : Filtrage des codes erreur et NULL

In [ ]:
%%sql
CREATE OR REPLACE TEMP VIEW consumption_clean AS
SELECT * FROM consumption_dedup
WHERE consumption_mw IS NOT NULL 
  AND consumption_mw >= 0
  AND consumption_mw < 10  -- Retirer valeurs aberrantes

## Cellule 4 : Vérification du nettoyage

In [ ]:
%%sql
SELECT 
    COUNT(*) as clean_rows,
    (SELECT COUNT(*) FROM bronze.consumption_raw) as original_rows,
    (SELECT COUNT(*) FROM bronze.consumption_raw) - COUNT(*) as removed_rows
FROM consumption_clean

## Cellule 5 : Normalisation des dates

In [ ]:
%%sql
CREATE OR REPLACE TEMP VIEW consumption_dates_fixed AS
SELECT 
    site_id,
    consumption_mw,
    voltage_v,
    frequency_hz,
    status,
    COALESCE(
        TRY_CAST(timestamp AS TIMESTAMP),
        TO_TIMESTAMP(timestamp, 'dd/MM/yyyy HH:mm:ss'),
        TO_TIMESTAMP(timestamp, 'yyyy-MM-dd HH:mm:ss')
    ) as timestamp_clean
FROM consumption_clean

## Cellule 6 : Agrégation horaire

In [ ]:
%%sql
CREATE OR REPLACE TEMP VIEW consumption_hourly AS
SELECT 
    DATE_TRUNC('hour', timestamp_clean) as hour,
    site_id,
    AVG(consumption_mw) as avg_consumption_mw,
    MAX(consumption_mw) as max_consumption_mw,
    MIN(consumption_mw) as min_consumption_mw,
    COUNT(*) as measurements
FROM consumption_dates_fixed
WHERE timestamp_clean IS NOT NULL
GROUP BY DATE_TRUNC('hour', timestamp_clean), site_id

## Cellule 7 : Jointure avec prix spot

In [ ]:
%%sql
CREATE OR REPLACE TEMP VIEW consumption_with_prices AS
SELECT 
    c.*,
    p.price_eur_mwh,
    p.market
FROM consumption_hourly c
LEFT JOIN bronze.market_prices p
  ON DATE_TRUNC('hour', c.hour) = DATE_TRUNC('hour', CAST(p.timestamp AS TIMESTAMP))

## Cellule 8 : ⚠️ LIMITE SQL - Logique métier complexe

### Problème rencontré

**Tâche** : Appliquer des règles métier complexes (calcul de baseline intelligente, z-score, interpolation)

**Limites de SQL** :
- ❌ Pas d'UDF (User Defined Functions) pour logique custom
- ❌ CASE WHEN devient illisible avec >10 règles
- ❌ Impossible d'implémenter des algorithmes itératifs
- ❌ Pas d'interpolation, pas de z-score simple

**💡 C'EST ICI QU'ON PASSE À PYSPARK**

Avec PySpark :
- ✅ UDF custom en Python (5 lignes)
- ✅ Logique métier lisible et maintenable
- ✅ Fonctions mathématiques avancées
- ✅ 10× plus rapide pour calculs complexes

→ **Notebook 3 : PySpark débloque tout ça**

## Cellule 9 : Sauvegarde partielle dans Silver

In [ ]:
# Sauvegarder ce qu'on a nettoyé
df = spark.sql("SELECT * FROM consumption_with_prices")
df.write.mode("overwrite").format("delta").saveAsTable("silver.consumption_clean")

print(f"✅ Table silver.consumption_clean créée : {df.count()} lignes")

## Cellule 10 : Résumé

### ✅ Nettoyage SQL terminé

**Ce qu'on a fait avec SQL** :
- ✅ Suppression doublons (~900 lignes)
- ✅ Filtrage NULL et codes erreur (~900 lignes)
- ✅ Normalisation formats de dates
- ✅ Agrégation horaire
- ✅ Jointure avec prix spot

**Ce qu'on NE PEUT PAS faire avec SQL** :
- ❌ Calcul baseline intelligente (exclure weekends + maintenance)
- ❌ Détection anomalies avec z-score
- ❌ Feature engineering pour ML
- ❌ UDF personnalisées

**Résultat** :
- ~16,300 lignes propres sauvegardées dans `silver.consumption_clean`

➡️ **Prochaine étape** : PySpark pour calculs avancés (Notebook 3)